In [ ]:
# File: noodle.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [276]:
import os
import random
import numpy as np

from frontend import App
app = App.create("noodle")


#bc = []
#bc.append((1.0, 2.5, 3.0))
#bc.append((0, 2.5, 3.0))
#bc.append((2, 2.5, 3.0))

bc = [
    [-0.20, 0.1, 0.60],
    [-0.14, 0.1, 0.29],
    [0.03, 0.1, 0.03],
    [0.29, 0.1, -0.14],
    [0.60, 0.1, -0.20],
    [0.91, 0.1, -0.14],
    [1.17, 0.1, 0.03],
    [1.34, 0.1, 0.29],
    [1.40, 0.1, 0.60],
    [1.34, 0.1, 0.91],
    [1.17, 0.1, 1.17],
    [0.91, 0.1, 1.34],
    [0.60, 0.1, 1.40],
    [0.29, 0.1, 1.34],
    [0.03, 0.1, 1.17],
    [-0.14, 0.1, 0.91]
]



model_root = "/tmp/objModels"
checkout_list = [
    "Models",
]
app.extra.sparse_clone(
    "https://github.com/FayetJ/FiberContactTest", model_root, checkout_list
)

bushing_path = os.path.join(model_root, checkout_list[0], "Bushing.obj")
tcp_end_path = os.path.join(model_root, checkout_list[0], "TcpEnd.obj")

V, E = app.mesh.line([0, 0.01, 0], [1.5, 25, 0], 960)
app.asset.add.rod("strand", V, E)
V, E = app.mesh.line([0, 0, 0], [0, 1, 0], 10)
app.asset.add.rod("repere", V, E)

scene = app.scene.create()
scene.add.invisible.wall([0, 0, 0], [0, 1, 0])


N, scale = 1, 0.05
x = -0.68
y = 3

bclen = len(bc)

#for i in range(bclen):
#    scene.add("repere").at(bc[i][0],bc[i][1],bc[i][2])


obj = scene.add("strand").at(x, 0, y).jitter()
(
    obj.param.set("bend", 6)
    .set("contact-gap", 3e-3)
    .set("contact-offset", 3e-3)
    .set("young-mod", 1e6)
    .set("friction", 0.02)
)

barpin = obj.pin(obj.grab([0,-1,0]))
#obj.pin(obj.grab([0,1,0])).move_by([0,-2,0],0.5)

V, F = app.mesh.load_tri(bushing_path)
app.asset.add.tri("bushing_body", V, F)

bushing_body = scene.add("bushing_body").at(0,0,0)
bushing_body.scale(0.05)
bushing_body.param.set("model","arap")

pin = bushing_body.pin()

V, F = app.mesh.load_tri(tcp_end_path)
app.asset.add.tri("tcp_end_body", V, F)

tcp_end_body = scene.add("tcp_end_body").at(-1.25,0.5,2.45)
tcp_end_body.scale(0.1)
tcp_end_body.param.set("model","arap")
pin2 = tcp_end_body.pin()

#barpin.move_by([0,0,-4],2)
barpin.interp("linear")

nCycles = 10
offset = 0.8/nCycles
timeOffset = 2
interpTimeOffset = 0.2

#barpin.move_to([[0.0,0,0]],3.0)
#print(V)

vlen = len(V)
offset=1

#for i in range(vlen):
#    V[i][0] = V[i][0]+1
#    V[i][2] = V[i][2]+1

pin2.move_by([0,1,0],3)

for j in range(nCycles):
    for i in range(bclen):
    #barpin.move_to([[bc[i][0],bc[i][1],bc[i][2]]],i)
        currentx = bc[i][0]
        currentz = bc[i][2]
    
        #pin2.move_to([[currentx-x,0.1+offset*j,currentz-y]],(i+j*bclen)*interpTimeOffset+timeOffset)

        
    #barpin.move_to([[0,0,0]],i)
    
#barpin.move_to([[0.0,0.3,-2.0]],1.0)
#barpin.move_to([[2,0.3,-4.0]],2.0)
#barpin.move_to([[0.0,0.0,10.0]],10.0)
#pin.move_by([0,0.1,0],5)

#opts = {"lookat": [0, 0.25, 0], "eyeup": 1.0, "fov": 4}
scene = scene.build().report()
scene.preview(options=opts)

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
build scene: 100%|███████████████████| 11/11 [00:00<00:00, 736.34it/s]


#vert,#rod,#tri,#pin,#static_vert,#static_tri
"1,569",960,380,609,908,604


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 25.00592041015625, 31.257400512695…

interactive(children=(FloatSlider(value=0.0, description='time', max=3.0, step=0.01), Output()), _dom_classes=…

In [277]:
session = app.session.create(scene)
(
    session.param.set("frames", 2400)
    .set("fix-xz", 1)
    .set("isotropic-air-friction", 1e-5)
    .set("gravity", 3)
    .set("dt",0.005)
)
session = session.build()

build session: 100%|█████████████████| 14/14 [00:00<00:00, 777.75it/s]


In [278]:
session.start().preview(options=opts)
session.stream()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 25.00592041015625, 31.257400512695…

HTML(value='')

HTML(value='')

In [83]:
session.animate(options=opts)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 50.00094985961914, 62.501186370849…


Loading frames: 100%|███████████████| 10/10 [00:00<00:00, 7444.63it/s]


interactive(children=(IntSlider(value=1, description='frame', max=10, min=1), Output()), _dom_classes=('widget…

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()